In [180]:
import numpy as np
from numpy import sqrt, log, exp, pi
import pandas as pd
import scipy.stats
from scipy.stats import norm

from math import sqrt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

from numpy import mean, absolute
import yfinance as yf

from scipy.optimize import minimize


import warnings
warnings.filterwarnings("ignore")


In [224]:
# Definindo as ações e o período de análise
acoes = ["PETR4.SA", "VALE3.SA", "ITUB4.SA", "GGBR4.SA", "BRL=X", "CSNA3.SA", "CMIG4.SA", "BBDC4.SA", "BBSE3.SA", "B3SA3.SA", "ABEV3.SA"]
inicio = "2016-01-01"
fim = "2016-12-31"

# Baixando os dados do Yahoo Finance
dados = yf.download(acoes, start = inicio, end = fim)["Adj Close"]

# Calculando os retornos diários
retornos = dados.pct_change().dropna()

# Calculando a média e o desvio padrão dos retornos
media_retornos = retornos.mean()
desvio_padrao = retornos.std()


[*********************100%***********************]  11 of 11 completed


In [225]:
# Criando o gráfico de dispersão com Plotly
fig = go.Figure()

# Adicionando os pontos ao gráfico
fig.add_trace(go.Scatter(
    x = desvio_padrao*100,
    y = media_retornos*100,
    mode = "markers+text",
    text = acoes,
    textposition = "top center"
))

# Configurando os títulos dos eixos
fig.update_layout(
    height = 800, width = 800,
    title = "Risco vs Retorno Esperado",
    xaxis_title = "Risco (Desvio Padrão)",
    yaxis_title = "Retorno Esperado"
)

fig.update_layout(hovermode = "x")


# Exibindo o gráfico
fig.show()


In [226]:
# Função para calcular retorno, volatilidade e Sharpe Ratio de um portfólio
def get_ret_vol_sr(weights):
    weights = np.array(weights/100)
    ret = np.sum(retornos.mean() * weights * 252)  # Retorno anualizado
    vol = np.sqrt(np.dot(weights.T, np.dot(retornos.cov()*252, weights)))  # Volatilidade anualizada
    sr = ret/vol  # Sharpe Ratio
    return np.array([ret, vol, sr])

# Função para minimização (negativo do Sharpe Ratio)
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1

# Restrições (soma dos pesos igual a 1)
cons = ({"type":"eq","fun": lambda x: np.sum(x) - 1})

# Limites para os pesos (entre 0 e 1 para cada ativo)
bounds = ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

# Palpite inicial (distribuição igualitária dos pesos)
init_guess = [1/len(acoes)]*len(acoes)

# Otimização para encontrar os pesos que maximizam o Sharpe Ratio
opt_results = minimize(neg_sharpe, init_guess, method = "SLSQP", bounds = bounds, constraints = cons)

# Pesos otimizados para o portfólio
optimal_weights = np.round(opt_results.x, 2)

print("Carteira: ", acoes)
print("Pesos Otimizados:", optimal_weights)


Carteira:  ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'GGBR4.SA', 'BRL=X', 'CSNA3.SA', 'CMIG4.SA', 'BBDC4.SA', 'BBSE3.SA', 'B3SA3.SA', 'ABEV3.SA']
Pesos Otimizados: [0.   0.37 0.17 0.   0.   0.   0.11 0.18 0.02 0.07 0.08]


In [227]:
carteira = acoes
pesos_otimizados = optimal_weights

# Combinando os tickers com seus respectivos pesos
combinados = zip(carteira, pesos_otimizados)

# Filtrando os tickers que têm peso maior que zero
acoes_com_peso = [ticker for ticker, peso in combinados if peso > 0]

print(acoes_com_peso)

# Filtrando para manter apenas os pesos não nulos
pesos_nao_nulos = [peso for peso in pesos_otimizados if peso > 0]

print(pesos_nao_nulos)

['VALE3.SA', 'ITUB4.SA', 'CMIG4.SA', 'BBDC4.SA', 'BBSE3.SA', 'B3SA3.SA', 'ABEV3.SA']
[0.37, 0.17, 0.11, 0.18, 0.02, 0.07, 0.08]


In [229]:
# Baixando dados históricos de 202X
dados = yf.download(acoes_com_peso, start = "2017-01-01", end = "2017-12-31")["Adj Close"]
dados_indice = yf.download("^BVSP", start = "2017-01-01", end = "2017-12-31")["Adj Close"]


# Calculando retornos diários
retornos_diarios = dados.pct_change().dropna()
retornos_diarios_indice = dados_indice.pct_change().dropna()


# Calculando retorno acumulado para a carteira com pesos otimizados
retorno_acumulado_otimizado = np.sum((retornos_diarios * pesos_nao_nulos).mean()) * 252

# Calculando o risco (desvio padrão) da carteira com pesos otimizados
risco_otimizado = np.sqrt(np.dot(pesos_nao_nulos, np.dot(retornos_diarios.cov() * 252, pesos_nao_nulos)))
risco_indice = retornos_diarios_indice.std() * np.sqrt(252)


# Considerando pesos iguais para todas as ações
pesos_iguais = [1/len(acoes_com_peso)] * len(acoes_com_peso)

# Calculando retorno acumulado para a carteira com pesos iguais
retorno_acumulado_igual = np.sum((retornos_diarios * pesos_iguais).mean()) * 252

# Calculando retorno acumulado e risco do índice
retorno_acumulado_indice = retornos_diarios_indice.mean() * 252
risco_indice = retornos_diarios_indice.std() * np.sqrt(252)

# Calculando o risco da carteira com pesos iguais
risco_igual = np.sqrt(np.dot(pesos_iguais, np.dot(retornos_diarios.cov() * 252, pesos_iguais)))

print(f"Retorno Acumulado com Pesos Otimizados: {np.round(retorno_acumulado_otimizado*100, 2)}")
print(f"Risco com Pesos Otimizados: {np.round(risco_otimizado*100, 2)}")
print(f"Retorno Acumulado com Pesos Iguais: {np.round(retorno_acumulado_igual*100, 2)}")
print(f"Risco com Pesos Iguais: {np.round(risco_igual*100, 2)}")
print(f"Retorno Acumulado do Índice Bovespa: {np.round(retorno_acumulado_indice*100, 2)}")
print(f"Risco do Índice Bovespa: {np.round(risco_indice*100, 2)}")

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
Retorno Acumulado com Pesos Otimizados: 30.54
Risco com Pesos Otimizados: 16.68
Retorno Acumulado com Pesos Iguais: 29.58
Risco com Pesos Iguais: 20.61
Retorno Acumulado do Índice Bovespa: 27.3
Risco do Índice Bovespa: 19.03


In [223]:
# Assumindo que "retornos" é um DataFrame com os retornos diários das ações
portfolio_retornos = retornos.dot(optimal_weights)

# Definindo o nível de confiança (por exemplo, 95%)
confianca = 0.05

# Calculando o VaR histórico
VaR_95 = portfolio_retornos.quantile(confianca)

print(f"Value at Risk (95% de confiança): {np.round(VaR_95*100, 2)}")


Value at Risk (95% de confiança): 1.83
